# Rafael Nunes Santana - 201800215

### Atividade 9 (Teste 2) - Pesquisa Operacional I

Nessa atividade estaremos implementando o algoritmo Simplex de duas fases com base no código feito para a atividade 8. Esse algoritmo trabalha com variáveis positivas e restrições do tipo A(i,:)*x <= b(i). Ele é capaz de resolver problemas de maximização e minimização e consegue resolver mais problemas que o algoritmo anterior pois consegue corrigir restrições que inicialmente não tem solução.

## Funcionamento do algoritmo

Relacionado à programação linear, que trabalha com funções do 1º grau, a ideia do algoritmo é bem simples. Inicialmente, atribui-se valor zero às variáveis, que seria distante da solução. Em seguida, incrementa-se pouco a pouco a variável que tem maior interferência positiva no resultado da função objetivo, ou seja, a que possui o maior coeficiente. Esta é chamada de "variável ativa" e tem grande importância inicial pois é a mais “lucrativa” delas, ou seja, a que mais nos aproxima da otimização.

Conforme este valor aumenta, o algoritmo testa todas as restrições, até que uma delas não seja satisfeita. Esta restrição recebe o nome de "restrição ativa". Neste momento, conhece-se o valor máximo da variável ativa. O procedimento, então, passa para a próxima variável que nos aproxima da boa solução, sempre levando em consideração o máximo valor que a primeira pôde atingir. A cada mudança destas, o Simplex converte todos os coeficientes (inclusive os da função objetivo) de acordo com os limites encontrados nas sucessivas restrições ativas.

O procedimento é repetido até que o incremento das variáveis apresente-se como um decréscimo do total atingido. Isto é identificado com o sinal negativo à frente dos coeficientes da função objetivo. Ao fim, os valores buscados serão conhecidos por meio de um sistema de equações, estas oriundas do problema inicial.

In [3]:
import numpy as np

In [110]:
# Argumentos da função:
# c: vetor com os coeficientes da função objetivo
# A: matriz com os coeficientes das restrições
# b: vetor com os lados direito das restrições
# minimizando: Se a função objetiva deve ser minimizada. As restrições passam a ser do tipo A(i,:)*x <= b(i)

# OBS.: estamos considerando todas as variáveis >= 0
def simplex(c, A, b, maximizando=True):
    n_variaveis = len(c)
    n_restricoes = len(A)
    
    # Criando e preenchendo a matriz tableau
    tableau = np.zeros((n_restricoes + 2, n_variaveis + n_restricoes + 1), dtype=float)
    tableau[-2, :n_variaveis] = [-i if maximizando else i for i in c]
    tableau[-1, n_variaveis:-1] = np.full(shape=n_restricoes, fill_value=1, dtype=float)
    
    for i in range(0, n_restricoes):
        tableau[i, :n_variaveis] = A[i]
        tableau[i, n_variaveis + i] = 1
        tableau[i, -1] = b[i]
        
    print("Tableau para o início da primeira fase:")
    print(tableau)

    # Passando o tableau para a forma padrão
    for i in range(0, n_restricoes):
        tableau[-1] -= tableau[i]

    print("\nTableau na forma padrão:")
    print(tableau)

    print("\nIniciando as iterações da primeira fase:")

    resultado_primeira_fase = fase_simplex(n_variaveis, n_restricoes, tableau, maximizando)

    print("\nIniciando as iterações da segunda fase:")

    return fase_simplex(n_variaveis, n_restricoes, tableau[:-1], maximizando)
    

def fase_simplex(n_variaveis, n_restricoes, tableau, maximizando):
    print(tableau, "\n")

    for coluna in range(0, n_variaveis):
        if tableau[-1, coluna] >= 0:
            continue

        indice_variavel_que_entra = coluna
        coeficientes_variavel_que_entra = tableau[0:n_variaveis - 1, indice_variavel_que_entra]
   
        #print("coluna elemento pivô:\n", indice_variavel_que_entra)
        termos_por_coeficientes = [tableau[i, -1] / tableau[i, indice_variavel_que_entra] for i in range(0, n_restricoes)]
        #print("termos por coeficientes pivô:\n", termos_por_coeficientes)

        indice_variavel_que_sai = None
        menor_valor = None

        for i in range(0, len(termos_por_coeficientes)):
            if termos_por_coeficientes[i] > 0 and (menor_valor is None or termos_por_coeficientes[i] < menor_valor):
                menor_valor = termos_por_coeficientes[i]
                indice_variavel_que_sai = i
        
        elemento_pivo = tableau[indice_variavel_que_sai, indice_variavel_que_entra]
             
        #print("elemento pivo:\n", elemento_pivo)
        #print("linha pivô:\n", indice_variavel_que_sai)
        
        nova_linha_pivo = tableau[indice_variavel_que_sai] / elemento_pivo
        #print("nova linha pivô:\n", nova_linha_pivo)
        
        tableau[indice_variavel_que_sai] = nova_linha_pivo
        
        # Recalculando a tableau
        for i in range(0, len(tableau)):
            if i == indice_variavel_que_sai:
                continue
                
            coeficiente_variavel_que_entra = tableau[i, indice_variavel_que_entra]
            #print(coeficiente_variavel_que_entra)
            
            tableau[i] = nova_linha_pivo * (-coeficiente_variavel_que_entra) + tableau[i]
            
        print(tableau, "\n")
        
    # coeficientes estão na ordem [x1, x2,..., xF1, xF2,...]
    coeficientes = []
    
    for j in range(0, n_variaveis + n_restricoes):
        encontrou_valor = False

        for i in range(0, n_restricoes + 1):
            if tableau[i, j] == 1:
                if encontrou_valor:
                    coeficientes.pop()
                    encontrou_valor = False
                    break

                coeficientes.append(tableau[i, -1])
                encontrou_valor = True
                
        if not encontrou_valor:
            coeficientes.append(0)
        
    return {'Z': tableau[-1, -1] if maximizando else -tableau[-1, -1], 'coeficientes': coeficientes}

In [111]:
A = [[2, 1, 2],
     [3, 3, 1]]

simplex([4, 1, 1], A, [4, 3], False)

Tableau para o início da primeira fase:
[[2. 1. 2. 1. 0. 4.]
 [3. 3. 1. 0. 1. 3.]
 [4. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0.]]

Tableau na forma padrão:
[[ 2.  1.  2.  1.  0.  4.]
 [ 3.  3.  1.  0.  1.  3.]
 [ 4.  1.  1.  0.  0.  0.]
 [-5. -4. -3.  0.  0. -7.]]

Iniciando as iterações da primeira fase:
[[ 2.  1.  2.  1.  0.  4.]
 [ 3.  3.  1.  0.  1.  3.]
 [ 4.  1.  1.  0.  0.  0.]
 [-5. -4. -3.  0.  0. -7.]] 

[[ 0.         -1.          1.33333333  1.         -0.66666667  2.        ]
 [ 1.          1.          0.33333333  0.          0.33333333  1.        ]
 [ 0.         -3.         -0.33333333  0.         -1.33333333 -4.        ]
 [ 0.          1.         -1.33333333  0.          1.66666667 -2.        ]] 

[[ 0.   -0.75  1.    0.75 -0.5   1.5 ]
 [ 1.    1.25  0.   -0.25  0.5   0.5 ]
 [ 0.   -3.25  0.    0.25 -1.5  -3.5 ]
 [ 0.    0.    0.    1.    1.    0.  ]] 


Iniciando as iterações da segunda fase:
[[ 0.   -0.75  1.    0.75 -0.5   1.5 ]
 [ 1.    1.25  0.   -0.25  0.5   0.5 ]
 [ 0. 

{'Z': 2.1999999999999997,
 'coeficientes': [0, 0.4000000000000001, 1.7999999999999998, 0, 0]}

In [112]:
A = [[1, 2, 0, 1],
     [1, 2, 1, 1],
     [1, 3, -1, 2],
     [1, 1, 1, 0]]

simplex([2, 6, 1, 1], A, [6, 7, 7, 5], False)

Tableau para o início da primeira fase:
[[ 1.  2.  0.  1.  1.  0.  0.  0.  6.]
 [ 1.  2.  1.  1.  0.  1.  0.  0.  7.]
 [ 1.  3. -1.  2.  0.  0.  1.  0.  7.]
 [ 1.  1.  1.  0.  0.  0.  0.  1.  5.]
 [ 2.  6.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  1.  1.  0.]]

Tableau na forma padrão:
[[  1.   2.   0.   1.   1.   0.   0.   0.   6.]
 [  1.   2.   1.   1.   0.   1.   0.   0.   7.]
 [  1.   3.  -1.   2.   0.   0.   1.   0.   7.]
 [  1.   1.   1.   0.   0.   0.   0.   1.   5.]
 [  2.   6.   1.   1.   0.   0.   0.   0.   0.]
 [ -4.  -8.  -1.  -4.   0.   0.   0.   0. -25.]]

Iniciando as iterações da primeira fase:
[[  1.   2.   0.   1.   1.   0.   0.   0.   6.]
 [  1.   2.   1.   1.   0.   1.   0.   0.   7.]
 [  1.   3.  -1.   2.   0.   0.   1.   0.   7.]
 [  1.   1.   1.   0.   0.   0.   0.   1.   5.]
 [  2.   6.   1.   1.   0.   0.   0.   0.   0.]
 [ -4.  -8.  -1.  -4.   0.   0.   0.   0. -25.]] 

[[  0.   1.  -1.   1.   1.   0.   0.  -1.   1.]
 [  0.   1.   0.   1.   0.   

{'Z': 11.0, 'coeficientes': [4.0, 0, 1.0, 2.0, 4.0, 0, 0.0, 0]}

In [116]:
A = [[3, 2],
     [4, 3],
     [1, 2]]

simplex([4, 1], A, [3, 6, 4], False)

Tableau para o início da primeira fase:
[[3. 2. 1. 0. 0. 3.]
 [4. 3. 0. 1. 0. 6.]
 [1. 2. 0. 0. 1. 4.]
 [4. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0.]]

Tableau na forma padrão:
[[  3.   2.   1.   0.   0.   3.]
 [  4.   3.   0.   1.   0.   6.]
 [  1.   2.   0.   0.   1.   4.]
 [  4.   1.   0.   0.   0.   0.]
 [ -8.  -7.   0.   0.   0. -13.]]

Iniciando as iterações da primeira fase:
[[  3.   2.   1.   0.   0.   3.]
 [  4.   3.   0.   1.   0.   6.]
 [  1.   2.   0.   0.   1.   4.]
 [  4.   1.   0.   0.   0.   0.]
 [ -8.  -7.   0.   0.   0. -13.]] 

[[ 1.          0.66666667  0.33333333  0.          0.          1.        ]
 [ 0.          0.33333333 -1.33333333  1.          0.          2.        ]
 [ 0.          1.33333333 -0.33333333  0.          1.          3.        ]
 [ 0.         -1.66666667 -1.33333333  0.          0.         -4.        ]
 [ 0.         -1.66666667  2.66666667  0.          0.         -5.        ]] 

[[ 1.5  1.   0.5  0.   0.   1.5]
 [-0.5  0.  -1.5  1.   0.   1.5]
 [-2.   

{'Z': 1.5, 'coeficientes': [0, 1.5, 0, 1.4999999999999998, 1.0]}

In [115]:
# Exemplo maximização
A = [[1, 1,  1],
     [2, 1, -1],
     [3, 2, -1]]

simplex([2, 3, 1], A, [40, 20, 30])

Tableau para o início da primeira fase:
[[ 1.  1.  1.  1.  0.  0. 40.]
 [ 2.  1. -1.  0.  1.  0. 20.]
 [ 3.  2. -1.  0.  0.  1. 30.]
 [-2. -3. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  1.  0.]]

Tableau na forma padrão:
[[  1.   1.   1.   1.   0.   0.  40.]
 [  2.   1.  -1.   0.   1.   0.  20.]
 [  3.   2.  -1.   0.   0.   1.  30.]
 [ -2.  -3.  -1.   0.   0.   0.   0.]
 [ -6.  -4.   1.   0.   0.   0. -90.]]

Iniciando as iterações da primeira fase:
[[  1.   1.   1.   1.   0.   0.  40.]
 [  2.   1.  -1.   0.   1.   0.  20.]
 [  3.   2.  -1.   0.   0.   1.  30.]
 [ -2.  -3.  -1.   0.   0.   0.   0.]
 [ -6.  -4.   1.   0.   0.   0. -90.]] 

[[  0.    0.5   1.5   1.   -0.5   0.   30. ]
 [  1.    0.5  -0.5   0.    0.5   0.   10. ]
 [  0.    0.5   0.5   0.   -1.5   1.    0. ]
 [  0.   -2.   -2.    0.    1.    0.   20. ]
 [  0.   -1.   -2.    0.    3.    0.  -30. ]] 

[[ -1.   0.   2.   1.  -1.   0.  20.]
 [  2.   1.  -1.   0.   1.   0.  20.]
 [ -1.   0.   1.   0.  -2.   1. -10.]
 [  4.   0

{'Z': 100.0, 'coeficientes': [0, 30.0, 10.0, 0, 100.0, -20.0]}

In [109]:
# Exemplo minimização:
A = [[1,  2, 2,  4],
     [2, -1, 1,  2],
     [4, -2, 1, -1]]

simplex([5, -4, 6, -8], A, [40, 8, 10], False)

Tableau para o início da primeira fase:
[[ 1.  2.  2.  4.  1.  0.  0. 40.]
 [ 2. -1.  1.  2.  0.  1.  0.  8.]
 [ 4. -2.  1. -1.  0.  0.  1. 10.]
 [ 5. -4.  6. -8.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  1.  0.]]

Tableau na forma padrão:
[[  1.   2.   2.   4.   1.   0.   0.  40.]
 [  2.  -1.   1.   2.   0.   1.   0.   8.]
 [  4.  -2.   1.  -1.   0.   0.   1.  10.]
 [  5.  -4.   6.  -8.   0.   0.   0.   0.]
 [ -7.   1.  -4.  -5.   0.   0.   0. -58.]]

Iniciando as iterações da primeira fase:
[[  1.   2.   2.   4.   1.   0.   0.  40.]
 [  2.  -1.   1.   2.   0.   1.   0.   8.]
 [  4.  -2.   1.  -1.   0.   0.   1.  10.]
 [  5.  -4.   6.  -8.   0.   0.   0.   0.]
 [ -7.   1.  -4.  -5.   0.   0.   0. -58.]] 

[[  0.     2.5    1.75   4.25   1.     0.    -0.25  37.5 ]
 [  0.     0.     0.5    2.5    0.     1.    -0.5    3.  ]
 [  1.    -0.5    0.25  -0.25   0.     0.     0.25   2.5 ]
 [  0.    -1.5    4.75  -6.75   0.     0.    -1.25 -12.5 ]
 [  0.    -2.5   -2.25  -6.75   0.     0.     

{'Z': -15.040000000000006,
 'coeficientes': [9.280000000000001, 12.96, 0, 1.1999999999999997, 0, 0, 0]}

In [83]:
# Outro exemplo de minimização:
A = [[3, 2, 1],
     [2, 5, 3]]

simplex([-2, -3, -4], A, [10, 15], False)

Tableau para o início da primeira fase:
[[ 3.  2.  1.  1.  0. 10.]
 [ 2.  5.  3.  0.  1. 15.]
 [-2. -3. -4.  0.  0.  0.]
 [ 0.  0.  0.  1.  1.  0.]]

Tableau na forma padrão:
[[  3.   2.   1.   1.   0.  10.]
 [  2.   5.   3.   0.   1.  15.]
 [ -2.  -3.  -4.   0.   0.   0.]
 [ -5.  -7.  -4.   0.   0. -25.]]

Iniciando as iterações da primeira fase:
[[ 2.2  0.  -0.2  1.  -0.4  4. ]
 [ 0.4  1.   0.6  0.   0.2  3. ]
 [-0.8  0.  -2.2  0.   0.6  9. ]
 [-2.2  0.   0.2  0.   1.4 -4. ]]


[[ 1.00000000e+00  0.00000000e+00 -9.09090909e-02  4.54545455e-01
  -1.81818182e-01  1.81818182e+00]
 [ 0.00000000e+00  1.00000000e+00  6.36363636e-01 -1.81818182e-01
   2.72727273e-01  2.27272727e+00]
 [ 0.00000000e+00  0.00000000e+00 -2.27272727e+00  3.63636364e-01
   4.54545455e-01  1.04545455e+01]
 [ 0.00000000e+00  0.00000000e+00  2.49800181e-16  1.00000000e+00
   1.00000000e+00 -4.44089210e-16]]



Iniciando as iterações da segunda fase:
[[ 1.          0.14285714  0.          0.42857143 -0.14285714  2.14

{'Z': -18.57142857142857,
 'coeficientes': [2.142857142857143, 0, 3.571428571428571, 0, 0]}